In [22]:
import numpy as np

import re # 텍스트 전처리

import csv, random

from gensim.models import KeyedVectors # 미리 훈련된 단어 벡터 읽기

In [2]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2019-05-11 16:51:16--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com... 52.216.233.117
Connecting to s3.amazonaws.com|52.216.233.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: 'GoogleNews-vectors-negative300.bin.gz'

GoogleNews-vectors- 100%[===================>]   1.53G   593KB/s    in 44m 47s 

2019-05-11 17:36:05 (599 KB/s) - 'GoogleNews-vectors-negative300.bin.gz' saved [1647046227/1647046227]



In [55]:
FILENAME = "GoogleNews-vectors-negative300.bin.gz"
w2v_mod = KeyedVectors.load_word2vec_format(FILENAME, binary=True, limit=300000)

In [230]:
# TODO get keywords
# 변수 설명
# idx: word에 해당하는 index에 대해서  motion 번호를 입력해둠
# word: tag 단어들
# path: 각 모션의 path, 추후 api 사용하게 될 시에 필요함

keywords = ['play', 'cellphone', 'game', 'teammates', 'knowing']
def most_relevant_tag(keywords):
    f = open('data.csv', 'r', encoding='utf-8')
    rdr = csv.reader(f)

    words = []
    path = []
    idx = []

    for i, line in enumerate(rdr):
        n = 0
        tmp = line[2:]
        for w in tmp:
            try:
                w2v_mod[w]
            except KeyError:
                pass
            else:
                if w not in words and w != '':
                    words.append(w)
                    idx.append([i])
                    n = n + 1
                elif w != '':
                    idx[words.index(w)].append(i)
        path.append(line[1])

    f.close

    W = []

    for word in words:
        try:
            W.append(w2v_mod[word])
        except KeyError:
            pass
    W_n = np.array(W) # tag에 있는 모든 단어들에 대해서 300차원의 vector를 모아둔 n * 300 차원의 행렬

    K = []

    for key in keywords:
        K.append(w2v_mod[key])

    K_n = np.array(K) # 문장에서 얻어낸 keyword의 vector를 모아둔 k* 300 차원의 행렬
    K_n = K_n.transpose() # W와 곱하기 위해서 transpose 해준 300 * k 행렬

    s_temp = np.matmul(W_n , K_n) # keyword에 대해 각각 유사도 점수를 나타내는 행렬

    S = np.sum(s_temp, axis=1) # 모든 keyword에 대해서 유사도를 합한 행렬
    m = np.argmax(S) # max index (can know which tag is the max)

    # 해당 index에 해당하는 모션이 많을 경우 랜덤으로 하나 뽑아줌
    t = random.sample(idx[m],1)

    # 해당 모션에 대해서 path를 받아옴
    p = path[t[0]]
    
    return t[0]